In [1]:
import re
import os
import sys
import time
import codecs
import random
import shutil
import smtplib
import requests
import datetime
import openpyxl
import configparser
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as BS
from pathlib import Path
from chardet import detect
from functools import reduce
# from datetime import datetime, date, timedelta
# from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [2]:
%config Completer.use_jedi = False

In [3]:
def _load_config():
    config_path = "./config.ini"
    with open(config_path, "rb") as ef:
        config_encoding = detect(ef.read())["encoding"]
    config = configparser.ConfigParser()
    config.read_file(codecs.open(config_path, "r", config_encoding))
    return config

In [4]:
config = _load_config()

In [5]:
def _requests_session():
    session = requests.session()
    headers = {
        "user-agent": config["Requests_header"]["user-agent"]
    }
    session.headers.update(headers)
    return session

In [6]:
def _requests_session():
    session = requests.session()
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36"
    }
    session.headers.update(headers)
    return session

In [7]:
def multiple_replace(sub_dict, text):
     # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, sub_dict.keys())))

     # For each match, look-up corresponding value in dictionary
    return regex.sub(lambda mo: sub_dict[mo.string[mo.start():mo.end()]], text)

In [8]:
def df_m_pick(df_path, col_name):
    df = pd.read_csv(df_path, encoding="utf-8-sig")
    df[df["資料年月"][0]] = df[col_name]
    df_m = df[["代號", "名稱", df["資料年月"][0]]]
    return df_m

In [9]:
market = ["sii", "otc", "rotc"]

### 路徑

In [10]:
os.makedirs("./sii_otc", exist_ok=True)

In [11]:
os.chdir("./sii_otc")
os.listdir()

['csv', 'xlsx']

In [12]:
# os.chdir("..")
# os.listdir()

In [13]:
# 過去raw csv
past_dir = "./csv/past"

# 過去上市櫃合併
past_all_dir = "./csv/past_all"

# 全部併成data
data_dir = "./csv/data"
data_rev_dir = "./csv/data/rev"
data_yoy_dir = "./csv/data/yoy"
data_cum_rev_dir = "./csv/data/cum-rev"
data_cum_yoy_dir = "./csv/data/cum-yoy"

# 單天csv
market_dir = "./csv/_market"
sii_dir = "./csv/_market/sii"
otc_dir = "./csv/_market/otc"
# rotc_dir = "./csv/_market/rotc"

# 單天合併
concat_dir = "./csv/all"

# 當天新增
add_dir = "./csv/add"

# 月營收xlsx
xlsx_dir = "./xlsx"

### 抓以前的資料

In [14]:
Y = list(range(111, 112))
M = list(range(8, 9))
mkt = ["sii", "otc"]

import itertools
cum = len(Y)*len(M)*len(mkt)

os.makedirs("./csv/past", exist_ok=True)

In [15]:
count = 0

for y, m, mk in itertools.product(Y, M, mkt):
    url = "https://mops.twse.com.tw/nas/t21/{}/t21sc03_{}_{}.csv".format(mk, y, m)
    rs = _requests_session()
    res = rs.get(url)
    res.encoding = "utf-8-sig"

    save_path = "./csv/past/m-rev-{}-{}-{}.csv".format(str(y+1911), str(m).zfill(2), mk)
    with open(save_path, "w", encoding="utf-8-sig") as save:
        save.write(res.text.replace("\r\n", "\n"))

    count += 1
    if count % 10 == 0:
        print("{} / {} done.".format(count, cum))
    time.sleep(random.uniform(1.5, 7))
    # print(url)
print("{} / {} done.".format(count, cum))

2 / 2 done.


### 下載當日rev

In [ ]:
def data_date(x):
    match = re.search("(\d+)/(\d+)", x)
    data_Y = int(match[1]) + 1911
    data_m = int(match[2])
    return datetime.datetime(data_Y, data_m, 1) + relativedelta(months=1) - datetime.timedelta(days=1)

In [ ]:
date_Y = date.today().year
date_Y

In [ ]:
if datetime.date.today().day < 15:
    data_m = datetime.date.today() - relativedelta(months=1)
else:
    data_m = date.today()
data_m.month

In [ ]:
# 測試改月
data_m = 3

In [ ]:
datetime.datetime.now().strftime("%Y-%m-%d-%H%M")

In [ ]:
def save_rev(market, date_Y, data_m):
    url = "https://mops.twse.com.tw/nas/t21/{}/t21sc03_{}_{}.csv".format(market, str(date_Y-1911), str(data_m))
    rs = _requests_session()
    res = rs.get(url)
    res.encoding = "utf-8-sig"
    mkdirs("./csv/_market/{}".format(market))

#     date_save = date.today().strftime("%Y-%m-%d")
    # date_save = yesterday.strftime("%Y-%m-%d")
    date_save = datetime.datetime.now().strftime("%Y-%m-%d-%H%M")

    save_path = "{}/{}/m-rev-{}-{}-{}-{}.csv".format(market_dir, market, str(date_Y), \
        str(data_m).zfill(2), market, datetime.datetime.now().strftime("%Y-%m-%d-%H%M"))
    with open(save_path, "w", encoding="utf-8-sig") as save:
        save.write(res.text.replace("\r\n", "\n"))

    return save_path

In [ ]:
df_path = [save_rev(i, date_Y, data_m) for i in market]
df_path

### 單日合併

In [ ]:
dfs = [pd.read_csv(path, encoding="utf-8-sig") for path in df_path]
df_c = pd.concat(dfs, axis=0, join="outer", ignore_index=True, copy=True)
df_cols = ["出表日期", "資料年月", "代號", "名稱", "產業", "月營收", "上月營收", \
           "去年當月營收", "MoM", "YoY", "累計營收", "去年累計", "累計YoY", "備註"]
df_c.columns = df_cols
drop_cols = ["出表日期", "上月營收", "去年當月營收", "去年累計"]
df_c.drop(drop_cols, axis=1, inplace=True)
df_c=df_c.drop_duplicates()
df_c["資料年月"] = df_c["資料年月"].apply(data_date)
df_c = df_c.sort_values(["YoY", "MoM"], ascending=False)

In [ ]:
mkdirs(concat_dir)
concat_path = "{}/m-rev-{}-{}-all-{}.csv".format(concat_dir, str(date_Y), str(data_m).zfill(2), datetime.datetime.now().strftime("%Y-%m-%d-%H%M"))
df_c.to_csv(concat_path, index=False, encoding="utf-8-sig")

In [ ]:
# dd_s = os.listdir("./csv/all-3/")

In [ ]:
# for a in dd_s:
#     path = "./csv/all-3/{}".format(a)
#     dd_ds = pd.read_csv(path, encoding="utf-8-sig")
#     dd_ds = dd_ds.drop_duplicates()
#     dd_ds.to_csv(path, index=False, encoding="utf-8-sig")

### 新增

In [ ]:
concat_list = list(Path(concat_dir).iterdir())
concat_list.sort()
concat_list

In [ ]:
last_path = concat_list[-1]
previous_path = concat_list[-2]

In [ ]:
previous_path.stem[:13]

In [ ]:
df_0 = pd.read_csv(last_path, encoding="utf-8-sig")
df_1 = pd.read_csv(previous_path, encoding="utf-8-sig")
df_0_1 = df_0.merge(df_1, on="代號", how="left", indicator=True)
df_0_not_1 = df_0_1[df_0_1["_merge"] == "left_only"].drop(columns=["_merge"])
df_0_not_1.columns = df_0_not_1.columns.str.replace("_x", "")
df_0_not_1.drop(list(df_0_not_1.filter(regex = "_")), axis = 1, inplace = True) 
df_0_not_1.fillna("", inplace=True)
df_0_not_1 = df_0_not_1.sort_values(["YoY", "MoM"], ascending=False, na_position="last")

In [ ]:
df_0_not_1.shape

In [ ]:
# mkdirs(add_dir)

In [ ]:
concat_list[-1].name.replace("all", "add")

In [ ]:
add_path = "{}/{}".format(add_dir, concat_list[-1].name.replace("all", "add"))
add_path

In [ ]:
df_0_not_1.to_csv(add_path, index=False, encoding="utf-8-sig")

### 併成xlsx

In [ ]:
add_list = list(Path(add_dir).iterdir())
concat_list = list(Path(concat_dir).iterdir())

In [ ]:
df_add = pd.read_csv(add_list[-1], encoding="utf-8-sig")
df_con = pd.read_csv(concat_list[-1], encoding="utf-8-sig")

In [ ]:
df_add = df_add.sort_values(["YoY", "MoM"], ascending=False, na_position="last")
df_con = df_con.sort_values(["YoY", "MoM"], ascending=False, na_position="last")

In [ ]:
add_list[-1].stem

In [ ]:
rev_m = re.search("\d+-\d+", add_list[-1].stem)[0]

In [ ]:
type(rev_m)

In [ ]:
rev_m_dtime = datetime.datetime.strptime(rev_m, "%Y-%m")

In [ ]:
previous_m = datetime.datetime.strftime(rev_m_dtime - relativedelta(months=1), "%Y-%m")

In [ ]:
previous_m

In [ ]:
data_yoy_list = list(Path(data_yoy_dir).iterdir())
df_data_yoy = pd.read_csv(data_yoy_list[-1], encoding="utf-8-sig")

In [ ]:
yoy_m = [i for i in df_data_yoy.columns if i.startswith(previous_m)][0]
yoy_m

MoM data

In [ ]:
data_rev_list = list(Path(data_rev_dir).iterdir())
data_rev_list

In [ ]:
df_data_rev = pd.read_csv(data_rev_list[-1], encoding="utf-8-sig", index_col="代號").drop(["名稱"], axis=1)

In [ ]:
df_data_mom = df_data_rev.pct_change(axis="columns", periods=1).apply(lambda x: x*100).round(decimals=2).reset_index(level=0)

In [ ]:
df_data_mom

In [ ]:
mom_m = [i for i in df_data_mom.columns if i.startswith(previous_m)][0]

In [ ]:
mom_m

In [ ]:
df_data_mom.columns

In [ ]:
previous_mom = df_data_mom[["代號", mom_m]]

In [ ]:
previous_mom.columns = ["代號", "上月MoM"]

In [ ]:
df_merge_mom = df_con.merge(previous_mom, on="代號", how="left", indicator=False)

In [ ]:
df_merge_mom

In [ ]:
# previous_yoy = df_yoy[["代號", yoy_m]]
# previous_yoy.columns = ["代號", "上月YoY"]

In [ ]:
# df_add_yoy = df_add.merge(previous_yoy, on="代號", how="left", indicator=False)

In [ ]:
# add_cols = df_add_yoy.columns.tolist()
# add_cols

In [ ]:
# add_cols.insert(-4, add_cols[-1])

In [ ]:
# add_cols[:-1]

In [ ]:
# df_add_yoy = df_add_yoy[add_cols[:-1]]

In [ ]:
# df_add_yoy[:1]

In [ ]:
def merge_previous_mom(df, df_mom):
    previous_mom = df_mom[["代號", mom_m]]
    previous_mom.columns = ["代號", "上月MoM"]
    df_merge_mom = df.merge(previous_mom, on="代號", how="left", indicator=False)
    mom_cols = df_merge_mom.columns.tolist()
    mom_cols.insert(-5, mom_cols[-1])
    return df_merge_mom[mom_cols[:-1]]

In [ ]:
def merge_previous_yoy(df, df_yoy):
    previous_yoy = df_yoy[["代號", yoy_m]]
    previous_yoy.columns = ["代號", "上月YoY"]
    df_merge_yoy = df.merge(previous_yoy, on="代號", how="left", indicator=False)
    yoy_cols = df_merge_yoy.columns.tolist()
    yoy_cols.insert(-4, yoy_cols[-1])
    return df_merge_yoy[yoy_cols[:-1]]

In [ ]:
df_add_yoy = merge_previous_yoy(df_add, df_data_yoy)
df_con_yoy = merge_previous_yoy(df_con, df_data_yoy)

In [ ]:
mkdirs(xlsx_dir)

In [ ]:
xlsx_path = "{}/{}.xlsx".format(xlsx_dir, add_list[-1].stem)
xlsx_path

In [ ]:
def df_to_xlsx(df, path, sht_name):
    df.to_excel(path, sheet_name=sht_name, index=False, float_format="%.2f", engine="openpyxl", encoding="utf-8-sig", freeze_panes=(1, 3))

In [ ]:
df_to_xlsx(df_add_yoy, xlsx_path, "新增")

In [ ]:
wb = openpyxl.load_workbook(filename=xlsx_path)
if rev_m in wb.sheetnames:
    del wb[rev_m]
wb.save(xlsx_path)
wb.close()

with pd.ExcelWriter(xlsx_path, engine="openpyxl", mode="a") as writer:
    df_to_xlsx(df_con, writer, rev_m)

In [ ]:
def html_tag(element, atr, value, text):
    return '<{} {}="{}">{}</{}>'.format(element, atr, value, text, element)

In [ ]:
def html_colorize(x):
    if x < 0:
        return html_tag("span", "style", "color:crimson;", "{}%".format(round(x)))
    elif x > 0:
        return html_tag("span", "style", "color:royalblue;", "{}%".format(round(x)))
    else:
        return "{}%".format(round(x))

In [ ]:
html_colorize(-5.78)

In [ ]:
html_colorize(0)

### 篩選

In [ ]:
df_add_yoy_c = df_add_yoy.copy()

In [ ]:
df_add_yoy_c[:1]

In [ ]:
df_add_yoy_c["yoy_chg"] = df_add_yoy_c["YoY"] - df_add_yoy_c["上月YoY"]

In [ ]:
df_add_yoy_c["YoY_html"] = df_add_yoy_c["YoY"].apply(html_colorize)

In [ ]:
df_add_yoy_c["YoY_html"]

In [ ]:
df_add_yoy_c["名稱代號"] = df_add_yoy_c["名稱"] + " " + df_add_yoy_c["代號"].astype(str) + " YoY " + df_add_yoy_c["YoY_html"]

In [ ]:
# df_add_yoy_c["名稱代號"] = df_add_yoy_c["名稱"] + " " + df_add_yoy_c["代號"].astype(str) + " YoY " + df_add_yoy_c["YoY"].round(0).astype(int).astype(str) + "%"

In [ ]:
df_add_yoy_c["名稱代號"] = df_add_yoy_c["名稱代號"].str.replace(" -", "-", regex=False)

In [ ]:
df_add_yoy_c["名稱代號"]

In [ ]:
yoy_chg_p_rank = df_add_yoy_c[df_add_yoy_c["yoy_chg"] > 0].sort_values(by=["yoy_chg"], ascending=False, na_position="last")["名稱代號"].tolist()

In [ ]:
yoy_chg_n_rank = df_add_yoy_c[df_add_yoy_c["yoy_chg"] < 0].sort_values(by=["yoy_chg"], ascending=True, na_position="last")["名稱代號"].tolist()

In [ ]:
yoy_chg_p_rank

In [ ]:
yoy_chg_n_rank

In [ ]:
if not df_add_yoy_c[(df_add_yoy_c["上月YoY"] < 0) & (df_add_yoy_c["YoY"] > 0)].empty:
    df_yoy_n_to_p = df_add_yoy_c[(df_add_yoy_c["上月YoY"] < 0) & (df_add_yoy_c["YoY"] > 0)]
    yoy_n_to_p = df_yoy_n_to_p.sort_values(by=["yoy_chg"], ascending=False, na_position="last")["名稱代號"].tolist()

In [ ]:
if not df_add_yoy_c[(df_add_yoy_c["上月YoY"] > 0) & (df_add_yoy_c["YoY"] < 0)].empty:
    df_yoy_p_to_n = df_add_yoy_c[(df_add_yoy_c["上月YoY"] > 0) & (df_add_yoy_c["YoY"] < 0)]
    yoy_p_to_n = df_yoy_p_to_n.sort_values(by=["yoy_chg"], ascending=True, na_position="last")["名稱代號"].tolist()

In [ ]:
yoy_rank = df_add_yoy_c.sort_values(by=["YoY"], ascending=False, na_position="last")["名稱代號"].tolist()

In [ ]:
yoy_rank

In [ ]:
list(reversed(yoy_rank))

In [ ]:
def pickup_filter(list_f, pick_num):
    if not list_f:
        return None
    if len(list_f) <= pick_num*2:
        pick = int(round(len(list_f)/3))
        return ", ".join(list_f[:pick])
    else:
        return ", ".join(list_f[:pick_num])   

In [ ]:
yoy_rank_str = pickup_filter(yoy_rank, 7)
yoy_rank_r_str = pickup_filter(list(reversed(yoy_rank)), 7)

In [ ]:
yoy_chg_p_rank_str = pickup_filter(yoy_chg_p_rank, 7)
yoy_chg_n_rank_str = pickup_filter(yoy_chg_n_rank, 7)

In [ ]:
if not df_add_yoy_c[(df_add_yoy_c["上月YoY"] < 0) & (df_add_yoy_c["YoY"] > 0)].empty:
    df_yoy_n_to_p = df_add_yoy_c[(df_add_yoy_c["上月YoY"] < 0) & (df_add_yoy_c["YoY"] > 0)]
    yoy_n_to_p = df_yoy_n_to_p.sort_values(by=["yoy_chg"], ascending=False, na_position="last")["名稱代號"].tolist()
else:
    yoy_n_to_p = []

if not df_add_yoy_c[(df_add_yoy_c["上月YoY"] > 0) & (df_add_yoy_c["YoY"] < 0)].empty:
    df_yoy_p_to_n = df_add_yoy_c[(df_add_yoy_c["上月YoY"] > 0) & (df_add_yoy_c["YoY"] < 0)]
    yoy_p_to_n = df_yoy_p_to_n.sort_values(by=["yoy_chg"], ascending=True, na_position="last")["名稱代號"].tolist()
else:
    yoy_p_to_n = []

In [ ]:
yoy_n_to_p

In [ ]:
rank_list = [yoy_rank, list(reversed(yoy_rank)), yoy_chg_p_rank, yoy_chg_n_rank, yoy_n_to_p, yoy_p_to_n]
rank_list

In [ ]:
yoy_rank_str, yoy_rank_r_str, yoy_chg_p_rank_str, yoy_chg_n_rank_str, yoy_n_to_p_str, yoy_p_to_n_str = \
    [pickup_filter(x, 7) for x in rank_list]

In [ ]:
print(yoy_chg_p_rank_str)
print(yoy_chg_n_rank_str)
print(yoy_n_to_p_str)
print(yoy_p_to_n_str)

mail text html

In [ ]:
mail_text ="""
<html>\
### 測試中<br/>\
<br/>\
月營收: {}<br/>\
<br/>\
更新時間: {}<br/>\
上次更新時間: {}<br/>\
<br/>\
YoY優: {}<br/>\
YoY差: {}<br/>\
<br/>\
YoY跳升: {}<br/>\
YoY下降: {}<br/>\
<br/>\
YoY轉正: {}<br/>\
YoY轉負: {}<br/>\
""".format(rev_m, update_text, previous_text, yoy_rank_str, yoy_rank_r_str, \
           yoy_chg_p_rank_str, yoy_chg_n_rank_str, yoy_n_to_p_str, yoy_p_to_n_str)

In [ ]:
soup_mail = BS(mail_text, "lxml")

In [ ]:
with open("R:/test.html", "w", encoding="utf-8") as save:
    save.write(soup_mail.prettify())

In [ ]:
mail_html = """<html>
<head></head>
<body><p></p></body>
</html>"""

In [ ]:
soup_mail = BS(mail_html, "lxml")

In [ ]:
soup_mail

In [ ]:
soup_mail.p.append("### 測試中")
soup_mail.p.append(soup_mail.new_tag("br"))
soup_mail.p.append("月營收: {}".format(rev_m))
soup_mail.p.append(soup_mail.new_tag("br"))
soup_mail.p.append("更新時間: {}".format(update_text))
soup_mail.p.append("上次更新時間: {}".format(previous_text))
soup_mail.p.append(soup_mail.new_tag("br"))
soup_mail.p.append("YoY優: {}".format(yoy_rank_str))
soup_mail.p.append("YoY差: {}".format(yoy_rank_r_str))
soup_mail.p.append(soup_mail.new_tag("br"))
soup_mail.p.append("YoY跳升: {}".format(yoy_chg_p_rank_str))
soup_mail.p.append("YoY下降: {}".format(yoy_chg_n_rank_str))
soup_mail.p.append(soup_mail.new_tag("br"))
soup_mail.p.append("YoY轉正: {}".format(yoy_n_to_p_str))
soup_mail.p.append("YoY轉負: {}".format(yoy_p_to_n_str))

In [ ]:
soup_mail

In [ ]:
BS(soup_mail.prettify(), "html.parser")

In [ ]:
with open("R:/test.html", "w", encoding="utf-8") as save:
    save.write(soup_mail.prettify())

mail text

In [ ]:
add_list = list(Path(add_dir).iterdir())

In [ ]:
add_list[-1].stem[-15:]

In [ ]:
add_list[-1].stem[:13]

In [ ]:
rev_m = re.search("\d+-\d+", add_list[-1].stem)[0]
rev_m

In [ ]:
update_time = datetime.datetime.strptime(add_list[-1].stem[-15:], "%Y-%m-%d-%H%M")
update_text = datetime.datetime.strftime(update_time, "%Y-%m-%d %H:%M")

In [ ]:
update_time = datetime.datetime.strptime(add_list[-1].stem[-15:], "%Y-%m-%d-%H%M")
update_text = datetime.datetime.strftime(update_time, "%Y-%m-%d %H:%M")

if add_list[-1].stem[:13] != add_list[-2].stem[:13]:
    previous_text = "無"
else:
    previous_time = datetime.datetime.strptime(add_list[-2].stem[-15:], "%Y-%m-%d-%H%M")
    previous_text = datetime.datetime.strftime(update_time, "%Y-%m-%d %H:%M")

In [ ]:
previous_text

In [ ]:
mail_text ="""
### 測試中

月營收: {}

更新時間: {}
上次更新時間: {}
""".format(rev_m, update_text, previous_text)
mail_text

In [ ]:
mail_text ="""
### 測試中

月營收: {}

更新時間: {}
上次更新時間: {}

YoY優: {}
YoY差: {}

YoY跳升: {}
YoY下降: {}

YoY轉正: {}
YoY轉負: {}
""".format(rev_m, update_text, previous_text, yoy_rank_str, yoy_rank_r_str, yoy_chg_p_rank_str, yoy_chg_n_rank_str, yoy_n_to_p_str, yoy_p_to_n_str)
mail_text

In [ ]:
def mail(attach_file=None):
    to_list = config["SMTP"]["to"].replace(" ", "").split(",")
    ccto_list = config["SMTP"]["ccto"].replace(" ", "").split(",")
    bccto_list = config["SMTP"]["bccto"].replace(" ", "").split(",")

    # Import the email modules
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.base import MIMEBase
    from email import encoders

    # Define email addresses to use
    addr_to = ",".join(to_list)    # 注意，不是分號
    addr_cc = ",".join(ccto_list)
    # addr_bcc = ",".join(bccto_list)
    addr_from = config["SMTP"]["from"]

    receive = to_list
    receive.extend(ccto_list)
    receive.extend(bccto_list)

    # Define SMTP email server details
    smtp_server = config["SMTP"]["smtp_server"]
    smtp_user   = config["SMTP"]["smtp_user"]
    smtp_pass   = config["SMTP"]["smtp_pass"]

    # Construct email
    msg = MIMEMultipart("alternative")
    
    msg["To"] = addr_to
    msg["CC"] = addr_cc
    msg["From"] = addr_from
    msg["Subject"] = "{}月營收-{}".format(attach_file.stem[11:13], re.search("\d{4}-\d{2}-\d{2}", attach_file.stem)[0])

    msg.attach(MIMEText(mail_text, "plain"))
    
    part = MIMEBase("application", "octet-stream")
    part.set_payload(open(file, "rb").read())
    encoders.encode_base64(part)
    part.add_header("Content-Disposition", "attachment; filename={}".format(attach_file.name))
    msg.attach(part)    

    # Attach parts into message container.
    # According to RFC 2046, the last part of a MIMEMultipart message, in this case
    # the HTML message, is best and preferred.
    msg.attach(part)

    # Send the message via an SMTP server
    s = smtplib.SMTP_SSL(smtp_server, 465)
    s.ehlo()
    s.login(smtp_user,smtp_pass)
    s.sendmail(addr_from, receive, msg.as_string())
    s.quit()
    print("Email sent!\n")

In [ ]:
xlsx_list = list(Path(xlsx_dir).iterdir())

In [ ]:
xlsx_list[-1].stem[11:13]

In [ ]:
re.search("\d{4}-\d{2}", xlsx_list[-1].stem)[0]

In [ ]:
re.search("\d{4}-\d{2}-\d{2}", xlsx_list[-1].stem)[0]

In [ ]:
xlsx_list[-1].stem[-10]

In [ ]:
attachment = xlsx_list[-1]

In [ ]:
mail(xlsx_list[-1])

一堆月營收上市櫃合併

In [16]:
def data_date(x):
    match = re.search("(\d+)/(\d+)", x)
    data_Y = int(match[1]) + 1911
    data_m = int(match[2])
    return datetime.datetime(data_Y, data_m, 1) + relativedelta(months=1) - datetime.timedelta(days=1)

In [17]:
past_list = list(Path("./csv/past").iterdir())
past_list[-1]

WindowsPath('csv/past/m-rev-2022-08-sii.csv')

In [18]:
past_sii = list(Path("./csv/past").glob("*sii*"))

In [19]:
past_otc = list(Path("./csv/past").glob("*-otc*"))

In [20]:
len(past_sii), len(past_otc)

(2, 2)

In [21]:
past_zip = [list(a) for a in zip(past_sii, past_otc)]

In [22]:
def past_concat(df_path):
    dfs = [pd.read_csv(path, encoding="utf-8-sig") for path in df_path]
    df_c = pd.concat(dfs, axis=0, join="outer", ignore_index=True, copy=True)
    df_cols = ["出表日期", "資料年月", "代號", "名稱", "產業", "月營收", "上月營收", "去年當月營收", "MoM", "YoY", "累計營收", "去年累計", "累計YoY", "備註"]
    df_c.columns = df_cols
    drop_cols = ["出表日期", "上月營收", "去年當月營收", "去年累計"]
    df_c.drop(drop_cols, axis=1, inplace=True)
    df_c=df_c.drop_duplicates()
    df_c["資料年月"] = df_c["資料年月"].apply(data_date)
    data_date_str = re.search("\d+-\d+", df_path[0].name)[0]
    concat_path = "{}/m-rev-{}.csv".format(past_all_dir, data_date_str)
    df_c.to_csv(concat_path, index=False, encoding="utf-8-sig")

In [23]:
a = list(map(past_concat, past_zip))

一大堆合併到一張表

In [24]:
past_all = list(Path("./csv/past_all").iterdir())

In [25]:
past_all.sort()

In [26]:
df_t_0_path = past_all[0]
df_t_0_path

WindowsPath('csv/past_all/m-rev-2016-01.csv')

In [27]:
past_all[-1]

WindowsPath('csv/past_all/m-rev-2022-08.csv')

In [28]:
df_t_s = past_all[1:]

In [29]:
def merge_data(dir_name, col_name):
    for file in list(Path("./csv/data/{}".format(dir_name)).iterdir()):
        file.unlink()
        
    df_m_0 = df_m_pick(df_t_0_path, col_name)
    df_m_0_path = "./csv/data/{}/m-rev-data-{}-{}.csv".format(dir_name, dir_name, df_m_0.columns[-1])
    df_m_0.to_csv(df_m_0_path, index=False, encoding="utf-8-sig")
    
    for df_t in df_t_s:
        df_temp_0_path = list(Path("./csv/data/{}".format(dir_name)).iterdir())[-1]
        df_temp_0 = pd.read_csv(df_temp_0_path, encoding="utf-8-sig")
    #     print(df_temp_0_path, df_temp_0.shape)

        df_temp_r = df_m_pick(df_t, col_name)
        df_temp_m = pd.merge(df_temp_0, df_temp_r, on=["代號", "名稱"], how="outer").drop_duplicates()
    #     print(df_t, df_temp_r.shape)

#         df_temp_m = df_temp_m.drop_duplicates()

    #     print(df_temp_m.shape)
        df_temp_m_path = "./csv/data/{}/m-rev-data-{}-{}.csv".format(dir_name, dir_name, df_temp_m.columns[-1])
        df_temp_m.to_csv(df_temp_m_path, index=False, encoding="utf-8-sig")

#         print(df_temp_0.shape, df_temp_r.shape, df_temp_m.shape)
#         print("{} done.\n".format(df_temp_m.columns[-1]))
        
    for file in list(Path("./csv/data/{}".format(dir_name)).iterdir())[:-1]:
        file.unlink()

In [30]:
merge_data("rev", "月營收")
merge_data("yoy", "YoY")
merge_data("cum-rev", "累計營收")
merge_data("cum-yoy", "累計YoY")

去除重複

In [31]:
def data_drop_dup(dir_name):
    path_rev = list(Path("./csv/data/{}".format(dir_name)).iterdir())[-1]
    df = pd.read_csv(path_rev, encoding="utf-8-sig")
    df_group = df.groupby(by=["代號"]).sum()
    df_name_last = df[["代號", "名稱"]].drop_duplicates(subset="代號", keep="last")
    df_data = pd.merge(df_group, df_name_last, on=["代號"], how="outer")
    df_data_cols = df_data.columns[:-1].insert(1, "名稱")
    df_data_fix = df_data[df_data_cols]
    df_data_fix.to_csv(path_rev, index=False, encoding="utf-8-sig")

In [32]:
dir_name_list = ["rev", "yoy", "cum-rev", "cum-yoy"]

In [33]:
for file in dir_name_list:
    data_drop_dup(file)

### 歷史比較

營收

In [ ]:
df_all_path = list(Path(concat_dir).iterdir())[-1]

In [ ]:
df_last = df_m_pick(df_all_path, "月營收")

In [ ]:
df_last.shape

In [ ]:
df_r_path = list(Path(data_rev_dir).iterdir())[-1]

In [ ]:
df_r = pd.read_csv(df_r_path, encoding="utf-8-sig")

In [ ]:
df_r.shape

In [ ]:
df_update = df_r.merge(df_last, on="代號", how="right", indicator=True)
df_update.columns = df_update.columns.str.replace("_x", "")
df_update.drop(list(df_update.filter(regex = "_")), axis = 1, inplace = True) 

In [ ]:
rank_dict = {
    1: "創高",
    2: "次高",
    3: "三高"
}
rank_dict

In [ ]:
def rank_des(x):
    if x in rank_dict.keys():
        return rank_dict[x]

In [ ]:
last_m = df_update.columns[-1]

In [ ]:
m_cols = [i for i in df_update.columns if re.match("\d+-\d+-\d+", i) is not None]
m_same_cols = [i for i in df_update.columns if i[5:7] == last_m[5:7]]

In [ ]:
df_update_cp = df_update.copy()

In [ ]:
df_update_cp["歷史"] = df_update[m_cols].rank(axis=1, method="max")[last_m].apply(rank_des)
df_update_cp["同月"] = df_update[m_same_cols].rank(axis=1, method="max")[last_m].apply(rank_des)
df_update_cp["歷史YoY"] = df_update[m_cols].pct_change(axis="columns", periods=12).rank(axis=1, method="max")[last_m].apply(rank_des)
df_update_cp["同月YoY"] = df_update[m_cols].pct_change(axis="columns", periods=12)[m_same_cols].rank(axis=1, method="max")[last_m].apply(rank_des)

In [ ]:
df_update_cp.fillna(np.nan, inplace=True)

In [ ]:
df_high = df_update_cp[["代號", "名稱", "歷史", "同月", "歷史YoY", "同月YoY"]]
df_high

股價

和新增併表

In [ ]:
add_path = list(Path(add_dir).iterdir())[-1]
add_path

In [ ]:
df_add = pd.read_csv(add_path, encoding="utf-8-sig")

In [ ]:
df_add_high = df_add.merge(df_high, on="代號", how="left", indicator=True)

In [ ]:
df_add_high.columns = df_add_high.columns.str.replace("_x", "")
df_add_high = df_add_high.drop(list(df_add_high.filter(regex = "_")), axis = 1, inplace = False).fillna(np.nan, inplace=False)

In [ ]:
xq_path = list(Path("D:/python/_mops/xq").iterdir())[-1]

In [ ]:
df_xq = pd.read_csv(xq_path, encoding="utf-8-sig").drop(["商品"],axis = 1, inplace = False)

In [ ]:
df_xq.columns = df_xq.columns.str.replace("代碼", "代號")

In [ ]:
df_add_high = df_add_high.merge(df_xq, on="代號", how="left", indicator=True)

In [ ]:
df_add_high = df_add_high.drop(list(df_add_high.filter(regex = "_")), axis = 1, inplace = False).fillna(np.nan, inplace=False)

In [ ]:
df_add_high = df_add_high[[c for c in df_add_high if c not in ["備註"]] + ["備註"]]

In [ ]:
df_add_high[:3]

In [ ]:
df_add_high[["MoM", "YoY", "累計YoY"]] = df_add_high[["MoM", "YoY", "累計YoY"]].round(2)

In [ ]:
df_add_high[:3]

In [ ]:
html_path = "R:/test.html"

In [ ]:
add_high_html = df_add_high.fillna("", inplace=False)

In [ ]:
add_high_html.to_html(html_path, index=False)